In [76]:
import sys
from os.path import abspath,join
sys.path.append(abspath(join('..', 'module')))

from filepath import *
from pandas.io import sql
from sqlalchemy import create_engine
from random import sample

import numpy as np

# Import dataframe into MySQL
database_username = 'root'
database_password = ''
database_ip       = '127.0.0.1'
database_name     = 'akupintar_db'
database_connection = create_engine('mysql+mysqlconnector://{0}:{1}@{2}/{3}'.
                                    format(database_username, database_password, 
                                           database_ip, database_name))

In [77]:
#Ini gak perlu
def generate_nilai(n_pendaftar):
    # Generate 100 nilai SAINTEK

    # I. Nilai Tes Potensi Skolastik, disesuaikan dengan nilain maksimum dan minimum dari rekapitulasi LTMPT
    # 1. Kemampuan Penalaran Umum
    k_penalaran = np.random.randint(98, 891, n_pendaftar).tolist()
    # 2. Kemampuan Kuantitatif
    k_kuantitaif = np.random.randint(244, 924, n_pendaftar).tolist()
    # 3. Pengetahuan & Pemahaman Umum
    pengetahuan_umum = np.random.randint(204, 848, n_pendaftar).tolist()
    # 4. Kemampuan Memahami Bacaan & Menulis
    baca_tulis = np.random.randint(73, 910, n_pendaftar).tolist()

    # II. Nilai Tes Kompetensi Akademik
    # 1. Matematika Saintek
    matematika_saintek = np.random.randint(219, 1123, n_pendaftar).tolist()
    # 2. Fisika
    fisika = np.random.randint(227, 1070, n_pendaftar).tolist()
    # 3. Kimia
    kimia = np.random.randint(213, 1067, n_pendaftar).tolist()
    # 4. Biologi
    biologi = np.random.randint(237, 1000, n_pendaftar).tolist()

    # Buat Dataframe
    newArray = {'k_penalaran': k_penalaran, 'k_kuantitaif': k_kuantitaif, 'pengetahuan_umum':pengetahuan_umum, 
                'baca_tulis':baca_tulis, 'matematika_saintek': matematika_saintek, 'fisika':fisika, 
                'kimia':kimia, 'biologi':biologi}
    data_nilai  = pd.DataFrame(newArray)
    
    #Perhitungan statistik
    data_nilai['rerata'] = data_nilai.mean(axis=1)
    data_nilai['userId'] = data_nilai.index
    
    return data_nilai

def generate_pilihan(n_user,n_jurusan):
    dict_user = {}

    for i in n_user:
        result = sample(range(0, n_jurusan), 2)
        dict_user[i] = result
        
    data = pd.DataFrame(dict_user).melt(var_name='userId',value_name='jurusanId')

    return data

In [78]:
#Generate Data Nilai
data_nilai = generate_nilai(10000)
data_pilihan = generate_pilihan(data_nilai.userId.tolist(),4)

# #save DB
data_nilai.to_sql(con=database_connection, name='rasionalisasi_nilai', if_exists='replace',index=False)
data_pilihan.to_sql(con=database_connection, name='rasionalisasi_pilihan', if_exists='append',index=False)